In [1]:
#re-write byte code processor
import easygui

def get_value(masterlist,mode_in,pointer):
    if mode_in == '0':
        return masterlist[pointer]
    if mode_in == '1':
        return pointer

def run_operation(masterlist,pointer_location,input_value=None):
    #this function 
    instruction_padded =  '0000' + str(masterlist[pointer_location]) 
    instruction = instruction_padded[-2:]
    if instruction=='99':
        return -1,'End',None
    par1 = instruction_padded[-3:-2]
    par2 = instruction_padded[-4:-3]
    par3 = instruction_padded[-5:-4]
    input1 = get_value(masterlist,par1,pointer_location + 1)
    input2 = get_value(masterlist,par2,pointer_location + 2)
    input3 = get_value(masterlist,par3,pointer_location + 3)
    if instruction=='01':
        masterlist[input3] = masterlist[input1] + masterlist[input2]
        return pointer_location + 4 , 'running' , None
    elif instruction=='02':
        masterlist[input3] = masterlist[input1] * masterlist[input2]
        return pointer_location + 4 , 'running' , None
    elif instruction=='03' and input_value==None:
        masterlist[input1] = int(easygui.enterbox("input parameter to location " + str(input1))) 
        return pointer_location + 2 , 'running' , None
    elif instruction=='03' and input_value!=None:
        masterlist[input1] = input_value 
        return pointer_location + 2 , 'Input Used', None
    elif instruction=='04':
        return pointer_location + 2 , 'Output Created' , masterlist[input1] 
    elif instruction=='05':
        if masterlist[input1] != 0:
            return masterlist[input2], 'running' , None
        elif masterlist[input1] == 0:
            return pointer_location + 3
    elif instruction=='06':
        if masterlist[input1] == 0:
            return masterlist[input2], 'running' , None
        elif masterlist[input1] != 0:
            return pointer_location + 3, 'running' , None
    elif instruction=='07':
        if masterlist[input1] < masterlist[input2]:
            masterlist[input3] = 1
        else: 
            masterlist[input3] = 0
        return pointer_location + 4, 'running' , None
    elif instruction=='08':
        if masterlist[input1] == masterlist[input2]:
            masterlist[input3] = 1
        else: 
            masterlist[input3] = 0
        return pointer_location + 4, 'running' , None
    else:
        return -1, 'ERROR ON POINTER ' + str(pointer_location), None

mastertest = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
#run_operation(mastertest,4)
#run_operation(mastertest,12)
sample_output = run_operation(mastertest,0,input_value=4)
print(sample_output)
print(mastertest)
sample_output[1]



(2, 'Input Used', None)
[3, 15, 3, 16, 1002, 16, 10, 16, 1, 16, 15, 15, 4, 15, 99, 4, 0]


'Input Used'

In [20]:
def run_amplifier_program(instructions_in,pointer,input_list,input_list_pointer=0):
    #this runs a program, will take an instruction in
    if instructions_in[-3:] != [99,99,99]:
        instructions_in.extend([99,99,99])
    program_outlist=[]
    pointer=0
    end_counter = 0
    input_value=input_list[input_list_pointer]
    while pointer >= 0 and pointer <= len(instructions_in) and end_counter <= len(instructions_in):
        pointer,state_change,output = run_operation(instructions_in,pointer,input_value=input_value)
        if state_change=='Input Used':
            if input_list_pointer >= len(input_list) - 1:
                input_list_pointer += 1
                input_value = None
            elif input_list_pointer < len(input_list) - 1:
                input_list_pointer += 1
                input_value = input_list[input_list_pointer]
        elif state_change=='Output Created':
            program_outlist.append(output)
        elif state_change=='End':
            break
        elif state_change[0:8] == 'ERROR ON':
            break
        end_counter =+ 1
    return pointer,input_list_pointer,program_outlist

test_programA = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
test_programB = test_programA.copy() 
test_programC = test_programA.copy()
test_programD = test_programA.copy()
test_programE = test_programA.copy()
test_phase = [4,3,2,1,0]

test_outA = run_amplifier_program(test_programA,pointer=0,input_list=[test_phase[0],0],input_list_pointer=0)
test_outB = run_amplifier_program(test_programB,pointer=0,input_list=[test_phase[1],test_outA[2][0]],input_list_pointer=0)
test_outC = run_amplifier_program(test_programC,pointer=0,input_list=[test_phase[2],test_outB[2][0]],input_list_pointer=0)
test_outD = run_amplifier_program(test_programD,pointer=0,input_list=[test_phase[3],test_outC[2][0]],input_list_pointer=0)
test_outE = run_amplifier_program(test_programE,pointer=0,input_list=[test_phase[4],test_outD[2][0]],input_list_pointer=0)
test_outE

(-1, 2, [43210])

In [23]:
def run_all_amplifiers(program,phase_list):
    programA = program.copy()
    programB = program.copy()
    programC = program.copy()
    programD = program.copy()
    programE = program.copy()

    outA = run_amplifier_program(programA,pointer=0,input_list=[phase_list[0],0],input_list_pointer=0)
    outB = run_amplifier_program(programB,pointer=0,input_list=[phase_list[1],outA[2][0]],input_list_pointer=0)
    outC = run_amplifier_program(programC,pointer=0,input_list=[phase_list[2],outB[2][0]],input_list_pointer=0)
    outD = run_amplifier_program(programD,pointer=0,input_list=[phase_list[3],outC[2][0]],input_list_pointer=0)
    outE = run_amplifier_program(programE,pointer=0,input_list=[phase_list[4],outD[2][0]],input_list_pointer=0)
    return outE[2][0]

test_program2 = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
run_all_amplifiers(test_program2,[1,0,4,3,2])

65210

In [24]:
to_try = [ list(map(int,list(str('0' + str(x))[-5:]))) for x in range(1234,43210) if '0' in str('0' + str(x))[-5:] and '1' in str('0' + str(x))[-5:] and '2' in str('0' + str(x))[-5:] and '3' in str('0' + str(x))[-5:] and '4' in str('0' + str(x))[-5:] ]

def find_max_setting(program):
    output_max = 0
    for p in to_try:
        output = run_all_amplifiers(program,p)
        if output > output_max:
            phase_max = p 
            output_max = max(output_max,output)
    return output_max,phase_max

print(find_max_setting([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]))
print(find_max_setting([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]))
print(find_max_setting([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]))

(43201, [4, 3, 2, 0, 1])
(54321, [0, 1, 2, 3, 4])
(65210, [1, 0, 4, 3, 2])


In [25]:
part1_input = [3,8,1001,8,10,8,105,1,0,0,21,42,67,84,109,122,203,284,365,446,99999,3,9,1002,9,3,9,1001,9,5,9,102,4,9,9,1001,9,3,9,4,9,99,3,9,1001,9,5,9,1002,9,3,9,1001,9,4,9,102,3,9,9,101,3,9,9,4,9,99,3,9,101,5,9,9,1002,9,3,9,101,5,9,9,4,9,99,3,9,102,5,9,9,101,5,9,9,102,3,9,9,101,3,9,9,102,2,9,9,4,9,99,3,9,101,2,9,9,1002,9,3,9,4,9,99,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99]

find_max_setting(part1_input)

(262086, [2, 1, 4, 0, 3])